<a href="https://colab.research.google.com/github/gigikenneth/mlops-wk1/blob/main/Gigi_%5BMLOps%5D_Week_1_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

### Problem

In the project this week, we will build a machine learning text classifier to predict news categories from the news article text. 
1. We will iterate on classification models with increasing level of complexity and improved performance. 
2. We will look at the impact of training data size on model performance.
3. [advanced] As an extension, you will explore how to augment data efficiently to your existing training data (efficiency measured as improvement in performance normalized by volume of data augmented). The reason to consider efficiency is that in most real-world scenarios, additional data is typically unlabeled. In order to augment it to your training data, you have to get it annotated which incurs some time & money (either yoursself or through one of the existing annotation tools such as Labelbox, Scale etc). 

Throughout the project there are suggested model architectures that we expect to work reasonably well for this problem. But if you wish to extend/modify any part of this pipeline, or explore new model architectures you should definitely feel free to do so.

### Deliverables
1. Train average word vector classifier and report model performance for training size = [500, 1000, 2000, 5000, 10000, 25000]
2. Train transformer encoder classifier and report model performance for training size = [500, 1000, 2000, 5000, 10000, 25000]
3. Report performance improvement on the test dataset from naive dataset augmentation outlined in the this notebook
4. [stretch] Experiment with advanced data augmentation techniques (a few ideas & pointers given in the notebook below)


## Step1: Prereqs & Installation

Download & Import all the necessary libraries we need throughout the project.

In [1]:
# Install all the required dependencies for the project

!pip install numpy
!pip install scikit-learn
!pip install gensim
!pip install sentence-transformers
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# Package imports that will be needed for this project

import numpy as np
import json
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer
from gensim.utils import tokenize as gensim_tokenizer
import gensim.downloader as gensim_downloader
from sklearn.base import BaseEstimator, TransformerMixin
from pprint import pprint
from sklearn.pipeline import Pipeline

# [TO BE IMPLEMENTED] 
# Add any other imports needed below depending on the model architectures you are using. For e.g.
# from sklearn.linear_model import LogisticRegression

In [29]:
# Global Constants
LABEL_SET = [
    'Business', 
    'Sci/Tech',
    'Software and Developement',
    'Entertainment',
    'Sports',
    'Health',
    'Toons',
    'Music Feeds'
]

WORD_VECTOR_MODEL = 'glove-wiki-gigaword-100'
SENTENCE_TRANSFORMER_MODEL = 'all-mpnet-base-v2'

TRAIN_SIZE_EVALS = [500, 1000, 2000, 5000, 10000, 25000]
#TRAIN_SIZE_EVALS = [500, 1000]
EPS = 0.001
SEED = 0

np.random.seed(SEED)

## Step 2: Download & Load Datasets 

[AG News](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html) is a collection of more than 1 million news articles gathered from more than 2000 news sources by an academic news search engine. The news topic classification dataset & benchmark was first used in [Character-level Convolutional Networks for Text Classification (NIPS 2015)](https://arxiv.org/abs/1509.01626). The dataset has the text description (summary) of the news article along with some metadata. **For this project, we will use a slightly modified (cleaned up) version of this dataset** 

Schema:
* Source - News publication source
* URL - URL of the news article
* Title - Title of the news article
* Description - Summary description of the news article
* Category (Label) - News category

Sample row in this dataset:
```
{
    'description': 'A capsule carrying solar material from the Genesis space '
                'probe has made a crash landing at a US Air Force training '
                'facility in the US state of Utah.',
    'id': 86273,
    'label': 'Entertainment',
    'source': 'Voice of America',
    'title': 'Capsule from Genesis Space Probe Crashes in Utah Desert',
    'url': 'http://www.sciencedaily.com/releases/2004/09/040908090621.htm'
 }
```




In [30]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

DIRECTORY_NAME = "data"
DOWNLOAD_URL = 'https://corise-mlops.s3.us-west-2.amazonaws.com/project1/agnews.zip'

def download_dataset():
    """
    Download the dataset. The zip contains three files: train.json, test.json and unlabeled.json 
    """
    http_response = urlopen(DOWNLOAD_URL)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=DIRECTORY_NAME)

# Expensive operation so we should just do this once
download_dataset()

In [31]:
Datasets = {}

for ds in ['train', 'test', 'augment']:
    with open('data/{}.json'.format(ds), 'r') as f:
        Datasets[ds] = json.load(f)
    print("Loaded Dataset {0} with {1} rows".format(ds, len(Datasets[ds])))

print("\nExample train row:\n")
pprint(Datasets['train'][0])

print("\nExample test row:\n")
pprint(Datasets['test'][0])

Loaded Dataset train with 25000 rows
Loaded Dataset test with 50000 rows
Loaded Dataset augment with 150000 rows

Example train row:

{'description': 'A capsule carrying solar material from the Genesis space '
                'probe has made a crash landing at a US Air Force training '
                'facility in the US state of Utah.',
 'id': 86273,
 'label': 'Entertainment',
 'source': 'Voice of America',
 'title': 'Capsule from Genesis Space Probe Crashes in Utah Desert',
 'url': 'http://www.sciencedaily.com/releases/2004/09/040908090621.htm'}

Example test row:

{'description': 'AP - Ellis L. Marsalis Sr., the patriarch of a family of '
                'world famous jazz musicians, including grandson Wynton '
                'Marsalis, has died. He was 96.',
 'id': 143852,
 'label': 'Entertainment',
 'source': 'Yahoo Entertainment',
 'title': 'Music Patriarch Marsalis Sr. Dies (AP)',
 'url': 'http://us.rd.yahoo.com/dailynews/rss/entertainment/*http://story.news.yahoo.com/news?tmpl

In [32]:
X_train, Y_train = [], []
X_test, Y_true = [], []
X_augment, Y_augment = [], []

for row in Datasets['train']:
    X_train.append(row['description'])
    Y_train.append(row['label'])

for row in Datasets['test']:
    X_test.append(row['description'])
    Y_true.append(row['label'])

for row in Datasets['augment']:
    X_augment.append(row['description'])
    Y_augment.append(row['label'])

## Step 3: [Modeling part 1] Word vectors

In [33]:
# Initialize the word vector model
word_vector_model = gensim_downloader.load(WORD_VECTOR_MODEL)

# Sanity check
print(word_vector_model.most_similar("cat"))
print(word_vector_model['cat'])

[('dog', 0.8798074722290039), ('rabbit', 0.7424426674842834), ('cats', 0.7323004007339478), ('monkey', 0.7288709878921509), ('pet', 0.7190139889717102), ('dogs', 0.7163872718811035), ('mouse', 0.6915250420570374), ('puppy', 0.6800068020820618), ('rat', 0.6641027331352234), ('spider', 0.6501135230064392)]
[ 0.23088    0.28283    0.6318    -0.59411   -0.58599    0.63255
  0.24402   -0.14108    0.060815  -0.7898    -0.29102    0.14287
  0.72274    0.20428    0.1407     0.98757    0.52533    0.097456
  0.8822     0.51221    0.40204    0.21169   -0.013109  -0.71616
  0.55387    1.1452    -0.88044   -0.50216   -0.22814    0.023885
  0.1072     0.083739   0.55015    0.58479    0.75816    0.45706
 -0.28001    0.25225    0.68965   -0.60972    0.19578    0.044209
 -0.31136   -0.68826   -0.22721    0.46185   -0.77162    0.10208
  0.55636    0.067417  -0.57207    0.23735    0.4717     0.82765
 -0.29263   -1.3422    -0.099277   0.28139    0.41604    0.10583
  0.62203    0.89496   -0.23446    0.5134

In [34]:
class WordVectorFeaturizer(BaseEstimator, TransformerMixin):
    def __init__(self, dim, word_vector_model):
        self.dim = dim
        self.word_vector_model = word_vector_model
        # you can add any other params to be passed to the constructor here
    
    #estimator. Since we don't have to learn anything in the featurizer, this is a no-op
    def fit(self, X, y=None):
        return self
    
    #transformation: return the average word vector of each token in the document
    def transform(self, X, y=None):
        """
        [TO BE IMPLEMENTED]
        
        Goal: WordVectorFeaturizer's transform() method converts the raw text document
        into a feature vector to be passed as input to the classifier.
            
        Given below is a dummy implementation that always maps it to a zero vector.
        You have to implement this function so it does two things:
            (1) Convert the raw text document into a list of tokens
            (2) Map each token to a word vector (using self.word_vector_model)
            (3) Return the mean vector of the sequence of word vectors
            *(4) Create vectors for words outside the vocabulary so it doesn't return an error
        This will be our feature representation of the document
        """

        X_t = []
        for doc in X:
          array = []
          #split the strings into individual words
          for element in doc.split():
            try:
              array.append(word_vector_model([element]))
            except:
              continue
          if len(array) != 0:
            X_t.append(np.mean(array, axis=0))
          else:
            X_t.append(np.zeros(self.dim))
            # TODO: replace this dummy implementation
            #X_t.append(np.zeros(self.dim))
        return X_t

In [35]:
from sklearn.linear_model import LogisticRegression
models = {}

for n in TRAIN_SIZE_EVALS:
    print("Evaluating for training data size = {}".format(n))
    X_train_i = X_train[:n]
    Y_train_i = Y_train[:n]

    """
    [TO BE IMPLEMENTED]
        
    Goal: initialized below is a dummy sklearn Pipeline object with no steps.
    You have to replace it with a pipeline object which contains at least two steps:
    (1) mapping the input document to a feature vector (using WordVectorFeaturizer)
    (2) a classifier that predicts the class label using the feature output of first step

    You can add other steps to preproces, post-process your data as you see fit. 
    You can also try any sklearn model architecture you want, but a linear classifier
    will do just fine to start with

    e.g. 
    pipeline = Pipeline([
        ('featurizer', <your WordVectorFeaturizer class instance here>),
        ('classifier', <your sklearn classifier class instance here>)
    ])
    """
    # pipeline =  Pipeline([
    #     'featurizer', WordVectorFeaturizer(dim=100,
    #                                        word_vector_model)
    
    #     'classifier', LogisticRegression
    # ])

    pipeline = Pipeline([('featurizer', WordVectorFeaturizer(dim=100, word_vector_model = word_vector_model)),
        ('classifier', LogisticRegression(solver='saga', tol=.001))])

    
    # train
    pipeline.fit(X_train_i, Y_train_i)
    # predict
    Y_pred_i = pipeline.predict(X_test)
    # record results
    models[n] = {
        'pipeline': pipeline,
        'test_predictions': Y_pred_i,
        'accuracy': accuracy_score(Y_true, Y_pred_i),
        'f1': f1_score(Y_true, Y_pred_i, average='weighted'),
        'errors': sum([x != y for (x, y) in zip(Y_true, Y_pred_i)])
    }
    print("Accuracy on test set: {}".format(accuracy_score(Y_true, Y_pred_i)))

Evaluating for training data size = 500
Accuracy on test set: 0.2431
Evaluating for training data size = 1000
Accuracy on test set: 0.27502
Evaluating for training data size = 2000
Accuracy on test set: 0.27502
Evaluating for training data size = 5000
Accuracy on test set: 0.2431
Evaluating for training data size = 10000
Accuracy on test set: 0.2431
Evaluating for training data size = 25000
Accuracy on test set: 0.27502


## Step 4: [Modeling part 2] Pretrained Transformers

In [36]:
# Initialize the pretrained transformer model
sentence_transformer_model = SentenceTransformer(
    'sentence-transformers/{model}'.format(model=SENTENCE_TRANSFORMER_MODEL))

# Sanity check
example_encoding = sentence_transformer_model.encode(
    "This is an example sentence",
    normalize_embeddings=True
)

print(example_encoding)


[ 2.25026179e-02 -7.82917142e-02 -2.30307244e-02 -5.10009658e-03
 -8.03404525e-02  3.91321592e-02  1.13428263e-02  3.46482964e-03
 -2.94574499e-02 -1.88930500e-02  9.47433487e-02  2.92747449e-02
  3.94859836e-02 -4.63165939e-02  2.54246984e-02 -3.21999528e-02
  6.21928386e-02  1.55591518e-02 -4.67794836e-02  5.03901616e-02
  1.46113932e-02  2.31413580e-02  1.22067202e-02  2.50696465e-02
  2.93655926e-03 -4.19821963e-02 -4.01039328e-03 -2.27843784e-02
 -7.68592861e-03 -3.31090540e-02  3.22118662e-02 -2.09992100e-02
  1.16730388e-02 -9.85074043e-02  1.77932634e-06 -2.29931921e-02
 -1.31140687e-02 -2.80222706e-02 -6.99970126e-02  2.59314254e-02
 -2.89502069e-02  8.76335949e-02 -1.20919021e-02  3.98605317e-02
 -3.31381895e-02  3.59108038e-02  3.46099287e-02  6.49784133e-02
 -3.00817527e-02  6.98188320e-02 -3.99512751e-03 -1.01597141e-03
 -3.50185595e-02 -4.36567590e-02  5.08026108e-02  4.68757935e-02
  5.39663583e-02 -4.03008610e-02  3.20132799e-03  1.36617851e-02
  3.82188521e-02 -3.23847

In [37]:
class TransformerFeaturizer(BaseEstimator, TransformerMixin):
    def __init__(self, dim, sentence_transformer_model):
        self.dim = dim
        self.sentence_transformer_model = sentence_transformer_model
        # you can add any other params to be passed to the constructor here

    #estimator. Since we don't have to learn anything in the featurizer, this is a no-op
    def fit(self, X, y=None):
        return self

    #transformation: return the encoding of the document as returned by the transformer model 
    def transform(self, X, y=None):
        X_t = []
        """
        [TO BE IMPLEMENTED]
        
        Goal: TransformerFeaturizer's transform() method converts the raw text document
        into a feature vector to be passed as input to the classifier.
            
        Given below is a dummy implementation that always maps it to a zero vector.
        You have to implement this function so it uses computes a document embedding
        of the input document using self.sentence_transformer_model. 
        This will be our feature representation of the document
        """
        for doc in X:
            # TODO: replace this dummy implementation
            X_t.append(self.sentence_transformer_model.encode(doc, normalize_embeddings=True))
            #X_t.append(np.zeros(self.dim))
        return X_t

In [ ]:
models_v2 = {}
for n in TRAIN_SIZE_EVALS:
    print("Evaluating for training data size = {}".format(n))
    X_train_i = X_train[:n]
    Y_train_i = Y_train[:n]

    """
    [TO BE IMPLEMENTED]
        
    Goal: initialized below is a dummy sklearn Pipeline object with no steps.
    You have to replace it with a pipeline object which contains at least two steps:
    (1) mapping the input document to a feature vector (using TransformerFeaturizer)
    (2) a classifier that predicts the class label using the feature output of first step

    You can add other steps to preproces, post-process your data as you see fit. 
    You can also try any sklearn model architecture you want, but a linear classifier
    will do just fine to start with

    e.g. 
    pipeline = Pipeline([
        ('featurizer', <your TransformerFeaturizer class instance here>),
        ('classifier', <your sklearn classifier class instance here>)
    ])
    """
    
    #pipeline = Pipeline()

    pipeline = Pipeline([('featurizer', TransformerFeaturizer(dim=100, sentence_transformer_model = sentence_transformer_model)),
        ('classifier', LogisticRegression(solver='saga', tol=.001))])


    # train
    pipeline.fit(X_train_i, Y_train_i)
    # predict
    Y_pred_i = pipeline.predict(X_test)
    # record results
    models_v2[n] = {
        'pipeline': pipeline,
        'test_predictions': Y_pred_i,
        'accuracy': accuracy_score(Y_true, Y_pred_i),
        'f1': f1_score(Y_true, Y_pred_i, average='weighted'),
        'errors': sum([x != y for (x, y) in zip(Y_true, Y_pred_i)])
    }
    print("Accuracy on test set: {}".format(accuracy_score(Y_true, Y_pred_i)))


Evaluating for training data size = 500
Accuracy on test set: 0.72456
Evaluating for training data size = 1000


## Step 5: Report Results from previous two steps

In [ ]:
# Report results

print("Word Vector Models: ")
for train_size, result in models.items():
    print("Train size: {0}  |  Accuracy: {1}  |  F1 score: {3} |  Num errors: {4}".format(
        train_size,
        result['accuracy'],
        result['f1'],
        result['errors']
    ))

print("Pretrained Transformer Models: ")
for train_size, result in models_v2.items():
    print("Train size: {0}  |  Accuracy: {1}  |  F1 score: {3} |  Num errors: {4}".format(
        train_size,
        result['accuracy'],
        result['f1'],
        result['errors']
    ))

## Step 6: Data Augmentation

In this section, we want to explore how to augment data efficiently to your existing training data. This is a very empirical exercise with a less well-defined playbook which means this section of the project is going to be open ended. Let us first understand what we mean by efficiency here, and why it matters:

### Performance Gain (G):
We will measure performance gain from data augmentation as the improvement in model accuracy (reduction in num. errors) on the Test dataset as defined above. 

### Budget (K):
We will measure "budget" as the number of additional rows augmentated to the original training dataset.  In this project, the universe of data from which you will select to add to your training set is Datasets['augment'] (and downstream X_augment, Y_augment).

This data is already labeled of course, but in most real-world scenarios the additional data is typically unlabeled. In order to augment it to your training data, you have to get it annotated which incurs some cost in time & money. This is the motivation to consider budget as a metric.

### Efficiency (E = G / K): 
Efficiency = Performance Gain (Reduction in num errors in test set) / Budget (Number of additional rows augmented to the training dataset)

We want to get the maximum gain in performance, while incurring minimum annotation cost.

In [ ]:
# Naively augmenting data by selecting (and incurring annotation cost) for K examples at random.

# In the code snippet below, we show the gain in performance from augmenting data naively
# at a few different budget values (K = 1000, 5000, 10000, 50000)

models_aug = {}

for K in [1000, 5000, 10000, 50000]:
    X_train_aug = X_train + X_augment[:K]
    Y_train_aug = Y_train + Y_augment[:K]

    """
    [TO BE IMPLEMENTED]
        
    Paste your pipeline created from Steps 3 or 4 here.
    (You can try both of them out to see what the performance gain is if any)

    e.g. 
    pipeline = Pipeline([
        ('featurizer', <your WordVectorFeaturizer class instance here>),
        ('classifier', <your sklearn classifier class instance here>)
    ])
    """
    pipeline =  Pipeline([('featurizer', WordVectorFeaturizer(dim = 100, word_vector_model = word_vector_model)),
    ('classifier', LogisticRegression(solver = 'saga',tol = 0.01))])

    # train
    pipeline.fit(X_train_aug, Y_train_aug)
    # predict
    Y_pred_i = pipeline.predict(X_test)
    # record results
    models_aug[K] = {
        'pipeline': pipeline,
        'test_predictions': Y_pred_i,
        'accuracy': accuracy_score(Y_true, Y_pred_i),
        'f1': f1_score(Y_true, Y_pred_i, average='weighted'),
        'errors': sum([x != y for (x, y) in zip(Y_true, Y_pred_i)])
    }
    print("Accuracy on test set: {}".format(accuracy_score(Y_true, Y_pred_i)))

### [Advanced] Suggested directions to explore to go beyond the naive augmentation:

Can we be more intelligent with the data we choose to augment to the training dataset? 

**Idea 1**: Look at the test errors that the current model is making. How can this help us guide our "data collection" for augmentation? One possible idea is to select examples from the augmentation dataset that are similar to these errors and add them to the training data. Similarity can be approximated in many ways:
1. [Jaccard distance between two texts](https://studymachinelearning.com/jaccard-similarity-text-similarity-metric-in-nlp/)
2. L2 distance between mean word vectors (we already compute these features for the entire dataset using WordVectorFeaturizer)
3. L2 distance between sentence transformer embedding (we already compute these features for the entire dataset using TransformerFeaturizer)
  

**Idea 2**: [Active Learning] Compute model's predictions on the augmentation dataset, and include those examples to the training dataset that the model finds "hard" ? (a proxy for this would be to look at cases where the output score distribution across all labels has nearly identical scores for top two or three labels). Some heuristics from this article might be helpful here: https://towardsdatascience.com/active-learning-in-machine-learning-525e61be16e5  

**Idea 3**: Look at the test errors that the current model is making, and the distribution of these errors across labels. Select examples from the augmentation dataset that belong to these classes - adding more training data for labels that the curent model does not do well on, can improve performance (assuming label quality is good)

In [ ]:
# Examine current test errors
test_errors = []
Y_pred_i = models[25000]['test_predictions']

for idx, label in enumerate(Y_true):
    if label != Y_pred_i[idx]:
        test_errors.append((X_test[idx], label,  Y_pred_i[idx]))

print("Number of errors in the test set: {}".format(len(test_errors)))
print("Example errors: [example, true label, predicted label]")
for i in range(10):
    print(test_errors[i])

In [ ]:
'''
[TO BE IMPLEMENTED]

Your additional data augmentation explorations go here

For instance, the pseudocode for Idea (1) might look like the following:

Augmented = {}
For e in test_errors:
   1. X_nn, y_nn = k nearest neighbors to (e) from X_augment, y_augment
   2. Add each (x, y) from (X_nn, y_nn) to Augmented

Add the Augmented examples to the training set
Train the new model and record performance improvements

'''